# Aula 5

## Problemas de alocação

Os problemas de alocação modelam uma relação entre dois tipos de entidades: *compromissos* e *recursos*. Adicionalmente, as restrições sobre a alocação de recursos a compromissos classificam-se em dois tipos: *obrigações* e *limitações*.

Nesta aula vamos considerar um problema de alocação. Pretende-se que faça a modelação do problema em Programação Inteira e que use o SCIP para os resolver.

In [ ]:
from pyscipopt import Model, quicksum

## Horário de um centro de estudos

Um centro de estudos possui $S$ salas de aula, $P$ professores, e está aberto $H$ horas por dia, durante $D$ dias na semana.

O centro de estudos funciona com as seguintes regras:
- Cada professor não pode dar mais do que $C$ horas por dia.
- Todos os professores do centro devem ter a mesma carga horária $T$ semanal.
- Não é permitido mais do que um professor por sala.
- Alocações contíguas do mesmo professor têm de ser na mesma sala.

Pretende-se estabelecer um horário para o centro de estudos que permita maximizar a carga horária de cada professor.

### Análise do problema

Este é um problema de alocação. Pretende-se alocar professores a salas de aula, ao longo da semana, sendo o tempo de ocupação das salas de uma hora. 

Existem $P$ professores, que podemos identificar por um índice $p \in [0..P\!-\!1]$, e podemos identificar cada sala disponível num dado dia, a uma dada hora, por um triplo $(s,d,h) \in [0..S\!-\!1]\times[0..D\!-\!1]\times[0..H\!-\!1]$.

Vamos usar uma família $x_{p,s,d,h}$ de variáveis binárias (i.e., que assumem valores inteiros $\{0,1\}$), com a seguinte semântica

$$x_{p,s,d,h} == 1  \quad \mbox{se e só se} \quad \mbox{o professor $p$ for alocado à sala $s$, no dia $d$, à hora $h$.}$$

Estas $P\times S\times D\times H$ variáveis são convenientemente representadas numa matriz $X$ instanciável com valores $\{0,1\}^{P\times S\times D\times H}$, a que se costuma chamar *matriz de alocação*.

Destaca-se ainda o seguinte:

**Limitações**

1. A carga horária diária máxima de cada docente é $C$.
2. Cada sala tem alocado, num máximo, um professor.
3. Em cada dia e hora, cada professor é alocado 0 ou 1 vezes.

**Obrigações**

4. Todos os professores têm a mesma carga horaria $T$ semanal.
5. No mesmo dia duas alocações contíguas do mesmo professor têm de ser na mesma sala.

**Objectivo**

Maximizar o número de horas de docência para cada professor.


### Implementação

Começamos por inicializar o modelo *horario* e definir os valores para as constantes $S$, $P$, $H$, $D$ e $C$.

In [ ]:
horario = Model()

S, P, H, D, C = 5, 8, 6, 5, 4

Em seguida, declaramos a matriz de alocação, $X$, e a variável $T$ que representa a carga horária semanal de cada professor.

### Exercício 1

Complete a declaração da matriz de alocação $X$ como um dicionário.

In [ ]:
x = {}


for p in range(P):
    x[p] = {}
    for s in range(S):
        x[p][s] = {}
        for d in range(D):
            x[p][s][d] = {}
            for h in range(H):
                x[p][s][d][h] = horario.addVar(str(p) + str(s) + str(d) + str(h), vtype = 'B')


def X(p,s,d,h):              # abreviatura
    return x[p][s][d][h]     # p = pessoas, s = sala, d = dia, h = hora      

T = horario.addVar("T",vtype="I",lb=0,ub=H*D) # lb = lower bound, ub = upper bound

Passamos agora à modelação das restrições e à sua introdução no SCIP.

A restrição

`1. A carga horária diaria máxima de cada docente é $C$`

pode expressar-se da seguinte forma:

$$\forall_{d< D}.\forall_{p< P}. \quad \big(\sum_{h< H,\,s< S} x_{p,s,d,h}\big) \leq C$$

In [ ]:
for d in range(D):
    for p in range(P):
        horario.addCons(quicksum([X(p,s,d,h) \
                         for h in range(H) for s in range(S)]) <= C)

### Exercício 2

Apresente as fórmulas que modelam as restantes restrições e acrescente-as ao modelo `horario`.

`2. Cada sala tem alocada, no máximo, um professor.`


pode expressar-se da seguinte forma:

$$\forall_{s< S}.\forall_{d< D}.\forall_{h< H}. \quad \big(\sum_{p< P} x_{p,s,d,h}\big) \leq 1$$

In [ ]:
for s in range(S): # para cada sala
    for d in range(D): # para cada dia
        for h in range(H): # para cada hora
            horario.addCons( quicksum( [ X(p,s,d,h) \
                           for p in range(P)] ) <= 1)

`3. Em cada dia e hora, cada professor é alocado 0 ou 1 vezes.`


pode expressar-se da seguinte forma:

$$\forall_{d< D}.\forall_{h< H}.\forall_{p< P}. \quad \big(\sum_{s< S} x_{p,s,d,h}\big) \leq 1$$

In [ ]:
 for d in range(D): # para cada dia
    for h in range(H): # para cada hora
        for p in range(P): # para cada professor
            horario.addCons( quicksum( [ X(p,s,d,h) \
                           for s in range(S)] ) <= 1)

`4. Todos os professores têm a mesma carga horaria $T$ semanal.`


pode expressar-se da seguinte forma:

$$\forall_{p< P}. \quad \big(\sum_{s< S,\,d< D,\,h< H} x_{p,s,d,h}\big) = T$$

In [ ]:
# completar
for d in range(D): # para cada dia
    horario.addCons( quicksum( [ X(p,s,d,h) \
                           for s in range(S) for d in range(D) for p in range(P)] ) <= T)

`5. No mesmo dia duas alocações contíguas do mesmo professor têm de ser na mesma sala.`


pode expressar-se da seguinte forma:

$$\forall_{d< D}.\forall_{p< P}.\forall_{h< H-1}.\forall_{s< s'< S} \quad \big( x_{p,s,d,h} + x_{p,s',d,h+1} \leq 1\big) \land \big( x_{p,s',d,h} + x_{p,s,d,h+1} \leq 1\big) $$

In [ ]:
# completar 
for d in range(D): # para cada dia
    for p in range(P): # para cada professor
        for h in range(H - 1): # para cada hora
            for s1 in range(S): # para cada sala
                for s2 in range(S-1):
                    horario.addCons(x[p][s1][d][h] + x[p][s2][d][h+1] <= 1)
                    horario.addCons(x[p][s2][d][h] + x[p][s1][d][h+1] <= 1)
                    
                

### Exercício 3

Finalize a resolução do problema, introduzindo o critério de optimização, procurando uma solução, e imprimindo-a, caso exista.

Recorde que  objectivo do problema é 
`Maximizar o número de horas de docência para cada professor.`



In [ ]:
# completar 
horario.setObjective(T, sense = 'maximize')
horario.optimize()

if horario.getStatus() == 'optimal':
    print(horario.getVal(T))

### Exercício 4

Defina funções para construir os horarios de professores e salas individuais, e para apresentar de forma legível esses horários.

In [ ]:
def prof(p):
   # completar 

def sala(s):
    # completar 

def apresenta(A):
    # completar             
            
            

print("\n=== Professor 0 ===")            
apresenta(prof(0))
print("\n=== Sala 0 ===")            
apresenta(sala(0))


### Exercício 5

Explore o comportamento do modelo pela variação dos parâmetros $S$, $P$, $H$, $D$ e $C$.

### Exercício 6

Queremos agora acrescentar a seguinte regra no funcionamento do centro de estudos:

`Cada professor tem de ter um dia da semana em que não dá aulas.`

Esta *obrigação* poderia ser expressa matemáticamente, de forma direta, por
$$
\forall_{p<P}.\exists_{d<D}. \quad \sum_{s<S,h<H} x_{p,s,d,h} = 0
$$
ou, em alternativa, pela seguinte expressão
$$
\forall_{p<P}. \quad \bigvee_{d<D} \big(\sum_{s<S,h<H} x_{p,s,d,h} = 0\big) 
$$

Contudo a disjunção não tem uma representação direta em SCIP. Para a implementar podemos
 acrescentar uma família de variáveis binárias $y_{p,d}$ que indicam se o professor $p$ dá aulas no dia $d$.
 O valor de cada $y_{p,d}$ será então a disjunção de todas as aulas do professor $p$, no dia $d$. O método para acrescentar esta restrição é o `addConsOr`, pelo que, para cada dia $d$, ter-se-á que invocar

   `horario.addConsOr([X(p,s,d,h) for s in range(S) for h in range(H)],y[p][d])`

Naturalmente que, para cada professor, a soma de todas estas variáveis $y_{p,d}$ terá que ser inferior a $D$.

Apresente as fórmulas que modelam esta nova obrigação. 

...

Acrescente agora as fórmulas ao modelo horario, e encontre nova solução.

**Nota:** Não é premitido modificar o problema depois de evocar o método `optimize`. Para reverter o solver para o estado antes de optimizar deverá usar método `freeTransform`.

In [ ]:
# completar 